<h1> VQE for $H_2$ molecule

In this tutorial, we use the Variational Quantum Eigensolver (VQE) algorithm to compute the ground-state energy of the Hydrogen gas molecule ($H_2$).
Particularly, we use `qiskit-alt` features to perform the Jordan-Wigner transform from the Fermionic operator that describes the $H_2$ molecule to a Pauli operator. 

In [1]:
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import X, Y, Z, I, PauliSumOp
from qiskit.circuit.library import TwoLocal

from qiskit import Aer

import numpy as np

import qiskit_alt, qiskit_alt.electronic_structure
from qiskit_alt.pauli_operators import PauliSum_to_SparsePauliOp
from qiskit.opflow import PauliSumOp

We start by simulating the $H_2$ molecule using the STO-3G basis:

In [2]:
# Describe the Hydrogen molecule by fixing the location of each nucleus
geometry = [['H', [0., 0., 0.]], ['H', [0., 0., 0.739]]]
basis = 'sto3g'

In order to map the problem to a quantum circuit, we first compute the Fermionic operator of the molecule, then map it to a Pauli operator using the Jordan-Wigner tranform:

In [3]:
# Compute the Fermionic operator of the molecule 
fermi_op = qiskit_alt.electronic_structure.fermionic_hamiltonian(geometry, basis)

# Convert the Fermionic operator to a Pauli operator using the Jordan-Wigner transform
pauli_op = qiskit_alt.electronic_structure.jordan_wigner(fermi_op);

# Convert the Pauli operator into a sum of Pauli operators
# input to the VQE algorithm to compute the minimum eigenvalue
pauli_sum_op = PauliSumOp(pauli_op)
print(pauli_sum_op)

-0.09577720253672618 * IIII
- 0.22389389642914415 * ZIII
+ 0.1715666759109326 * IZII
+ 0.12068268831019151 * ZZII
- 0.22389389642914415 * IIZI
+ 0.17445893297586684 * ZIZI
+ 0.16597132363812456 * IZZI
+ 0.04528863532793307 * XXXX
+ 0.04528863532793307 * YYXX
+ 0.04528863532793307 * XXYY
+ 0.04528863532793307 * YYYY
+ 0.17156667591093255 * IIIZ
+ 0.16597132363812456 * ZIIZ
+ 0.16873669065193608 * IZIZ
+ 0.12068268831019151 * IIZZ


We choose the ansatz and the optimizer, and run the the algorithm for 500 shots:

In [4]:
# Set up the ansatz of type TwoLOcal
ansatz = TwoLocal(num_qubits=4, 
                  rotation_blocks=['ry'], 
                  entanglement_blocks='cx', 
                  reps=1,
                  entanglement='linear', 
                  skip_final_rotation_layer= False)

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500)

We select the Aer backend and run the VQE algorithm according to the set parameters. 
We determine the value of the ground-state energy of the Hydrogen molecule as the minimum eigenvalue

In [5]:
# Select the backend for the quantum_instance
backend = Aer.get_backend('aer_simulator_statevector')

# Run VQE algorithm
vqe = VQE(ansatz = ansatz, 
          optimizer = optimizer,
          quantum_instance = backend)

# Compute the ground-state energy of the molecule
result = vqe.compute_minimum_eigenvalue(operator=pauli_sum_op)
print("The ground-state energy of the Hydrogen molecule is {} Hatree".format(round(result.eigenvalue.real,3)))

The ground-state energy of the Hydrogen molecule is -1.117 Hatree
